In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

In [47]:
isolados = pd.read_csv('data/01 - geral_denormalizada.csv')
isolados.sample(5)

,id,ano,ccr,amostra,coleta,latitude,longitude,data_coleta,localidade,equipe,substrato,nome_cientifico,cidade,uf
754,636,2015,7874,STL.TAQ.001.001,STL.TAQ.001,-1.031.507,-4.815.651,2015-01-11,"Distrito de Taquarucu, Palmas - TO",Stella Barros,Folha de Babaçu,Attalea speciosa,Palmas,TO
113,159,2014,7776,ANA.CAN.001.011,ANA.CAN.001,-9.978.136,-5.004.615,2014-01-24,Centro de Pesquisa Canguçú (Pium-TO),Ana Paula Guimarães,Fruto de Buriti,Mauritia flexuosa,Pium,TO
358,505,2014,7968,ANA.TAG.001.035,ANA.TAG.001,-12.392.825,-46.418.475,2014-01-31,Municipio de Taguatinga – TO,Ana Paula Guimarães,Fruto de Buriti,Mauritia flexuosa,Taguatinga,TO
3,61,2015,8560,JHF.BA.001.002,JHF.BA.001,-14.717.639,-52.363.527,2015-08-13,Córrego Bacaba,Jheferson Paulino,Fruto de Buriti,Mauritia flexuosa,Nova Xavantina,MT
300,290,2014,7153,ANA.TAG.001.017,ANA.TAG.001,-12.392.825,-46.418.475,2014-01-31,Municipio de Taguatinga – TO,Ana Paula Guimarães,Fruto de Buriti,Mauritia flexuosa,Taguatinga,TO


In [48]:
df = pd.read_csv('data/02 - reacoes_denormalizada.csv', names=['Ano','CCR','Composto','Resultado'], header=None, index_col=0)
df.sample(5)

,Ano,CCR,Composto,Resultado
61,2015,8560,2,+
98,2014,7802,65,-
643,2015,7914,13,-
208,2014,7021,64,-
573,2014,7533,13,-


In [54]:
# junta os termos em um unico identificador do isolado:
df["Isolado"] = "UFT."+df['Ano'].astype(str)+"(L)"+df['CCR'].astype(str)

# remove as colunas que não serao mais uteis:
del df['Ano']
del df['CCR']

In [60]:
# define nova coluna de indice:
df.set_index(['Isolado'], inplace=True)

In [70]:
# compostos = []
# for i in range(1,81):
#     compostos.append(i)

# compostos
df.sample(5)

,Composto,Resultado
Isolado,,
UFT.2015(L)7580,49,+
UFT.2015(L)8545,12,-
UFT.2014(L)6759,8,-
UFT.2014(L)7941,22,+
UFT.2014(L)7505,6,+


In [76]:
# converte coluna de resultados em linhas:
df = df.pivot(columns='Composto')

In [78]:
# salva estado atual do dataframe em arquivo CSV:
df.to_csv('03 - reacoes_formatadas.csv')

In [ ]:
# faz a transposição de coluna para linhas, deixando a tabela com a estrutura desejada:
df2.melt(id_vars=["", "name", ''], 
        var_name="Date", 
        value_name="Value")